In [1]:
# 依赖：
# !pip install sentencepiece
# !pip install jieba
# !pip install regex
# !pip install tensorflow
# !pip install tensorflow-hub

In [2]:
import tensorflow_hub as hub
import tensorflow as tf

from gpt2_tokenizer import GPT2Tokenizer

In [3]:
print(tf.__version__)

2.6.0


In [4]:
print(hub.__version__)

0.12.0


In [5]:
tokenizer = GPT2Tokenizer(
    'CPM-Generate/bpe_3w_new/vocab.json',
    'CPM-Generate/bpe_3w_new/merges.txt',
    model_file='CPM-Generate/bpe_3w_new/chinese_vocab.model')

In [6]:
gpt = hub.load('./cpm-distill-tf2/')

In [7]:
def sample(tokenizer, gpt, sentence, number=1, length=20, top_k=50, top_p=0.9, temperature=0.9):
    """
    numbert: 输出句子个数
    length: 输出最大长度
    top_k: 只选择前k个，如果为0则为无限制
    top_p: token的概率排在这以上才有效
    temperature: 温度，如果为1.0则相当于无效
    """
    inputs = tf.constant([tokenizer.encode(sentence)] * number, dtype=tf.int64)
    length = tf.constant(length, dtype=tf.int64)
    ret = gpt.signatures['serving_default'](
        inp=inputs,
        length=length,
        top_k=tf.constant(top_k, tf.int32),
        top_p=tf.constant(top_p, tf.float32),
        temperature=tf.constant(temperature, tf.float32)
    )['output_0']
    return [
        tokenizer.decode(s).replace(' ', '')
        for s in ret.numpy()
    ]

# 问答例子

In [8]:
def ask_gpt(question):
    q = f'''
问题：中国首都是哪里？
答案：北京
问题：美国的首都是哪里？
答案：华盛顿
问题：李白在哪个朝代？
答案：唐朝
问题：{question}
答案：'''
    ret = sample(tokenizer, gpt, q, 3, 10, top_k=50, top_p=0.9, temperature=0.9)
    answers = []
    for x in ret:
        a = x[len(q):]
        a = a.split('\n')[0]
        answers.append(a)
    return answers

In [9]:
ask_gpt('唐伯虎是哪个朝代的人？')

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 1.074 seconds.
Prefix dict has been built successfully.


['宋朝', '宋朝', '明朝']

In [10]:
ask_gpt('世界上最帅的是谁？')

['唐太宗', '中国著名的帝王', '毛泽东']

In [11]:
ask_gpt('世界上最聪明的人是谁？')

['希腊', '德国', '罗马']

In [12]:
ask_gpt('李清照是哪个朝代的人？')

['白族', '清代', '清朝']

In [13]:
ask_gpt('关公和秦琼谁厉害？')

['宋江', '赵飞燕和李逵谁厉害?', '唐朝']

In [14]:
ask_gpt('如何赚一百万美元？')

['华盛顿', '法国', '美国的首都是哪里?']

In [15]:
ask_gpt('烧烤和火锅哪个好吃？')

['⁇鱼', '北京', '川菜']

In [16]:
ask_gpt('如何成为老师？')

['李白', '孔子', '唐诗']

In [17]:
ask_gpt('佩奇是什么动物？')

['“乌龟”', '俄罗斯', '孔雀']

In [18]:
ask_gpt('大象有几条腿？')

['美国', '印度', '唐朝']

In [19]:
ask_gpt('姚明有多高？')

['姚明能登上奥运会!', '刘翔和刘翔的差距', '姚明']

In [20]:
ask_gpt('汤姆和杰瑞谁厉害？')

['约翰逊', '谁厉害?', '“杰克”']

In [21]:
ask_gpt('野原新之助能打过樱桃子吗？')

['曹操', '日本', '李白的诗写得很好']

In [22]:
ask_gpt('学音乐和学画画哪个好？')

['中国美术馆', '《四平调》', '钢琴']

In [23]:
ask_gpt('中文和英文哪个难学？')

['中文', '明朝', '中文和英文哪个容易?']

In [24]:
ask_gpt('python和java哪个难？')

['java', 'python', 'yahoo']

In [25]:
ask_gpt('“锄禾日当午”的下一句是什么？')

['唐', '“君子不器”', '“锄禾日当午”的']

In [26]:
ask_gpt('烤鸭的主要原料是什么？')

['烤鸭的主要原料是什么?', '生食', '新疆']

In [27]:
ask_gpt('把大象放冰箱总共分几步？')

['皇帝', '唐朝', '美国的首都是哪里?']

In [28]:
ask_gpt('珠穆朗玛峰有多高？')

['中国南边有多远?', '青藏高原有多高?', '香港']

In [29]:
ask_gpt('世界上最高的山是什么？')

['月球', '埃及', '美国的最高峰是巴黎']

In [30]:
ask_gpt('中国有多少人？')

['日本', '北京', '《史记》']

In [31]:
ask_gpt('日本有多少人？')

['元朝', '明朝', '中华民国']

In [32]:
ask_gpt('苹果手机好还是华为手机好？')

['三星', '三星', '努比亚']

# 推理

In [33]:
ret = sample(tokenizer, gpt, '小明决定去吃饭，小红继续写作业\n问题：去吃饭的人是谁？\n答案：', 3, 10, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

小明决定去吃饭,小红继续写作业
问题:去吃饭的人是谁?
答案:那个人是你朋友,小红想帮他
--------------------
小明决定去吃饭,小红继续写作业
问题:去吃饭的人是谁?
答案:是:小红。
问题二
--------------------
小明决定去吃饭,小红继续写作业
问题:去吃饭的人是谁?
答案:一个叫朱一超的小男孩
问题
--------------------


# 英文问答例子

In [34]:
ret = sample(tokenizer, gpt, '默写英文：\n狗dog\n猫cat\n鸟', 3, 10, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

默写英文:
狗dog
猫cat
鸟cat
羊cat
......
--------------------
默写英文:
狗dog
猫cat
鸟cat
鼠cat

--------------------
默写英文:
狗dog
猫cat
鸟grand
秃鹫grand
--------------------


# 默写古诗例子

In [35]:
ret = sample(tokenizer, gpt, '默写古诗：\n白日依山尽，黄河入海流。\n床前明月光，', 3, 10, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

默写古诗:
白日依山尽,黄河入海流。
床前明月光,疑是地上霜。
夜来幽
--------------------
默写古诗:
白日依山尽,黄河入海流。
床前明月光,疑是地上霜。
举头望
--------------------
默写古诗:
白日依山尽,黄河入海流。
床前明月光,疑是地上霜。
谁道
--------------------


# 不同的角色对话生成例子

In [36]:
ret = sample(tokenizer, gpt, '李大嘴：“各回各家，各找各妈！” \n佟掌柜：“', 3, 20, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

李大嘴:“各回各家,各找各妈!”
佟掌柜:“谁家没事?”
佟掌柜:“谁家没
--------------------
李大嘴:“各回各家,各找各妈!”
佟掌柜:“......”
杨大嘴:“我这个老太太
--------------------
李大嘴:“各回各家,各找各妈!”
佟掌柜:“唉!”
佟掌柜:“您可知道小老
--------------------


In [37]:
ret = sample(tokenizer, gpt, '李大嘴：“各回各家，各找各妈！” \n白展堂：“', 3, 20, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

李大嘴:“各回各家,各找各妈!”
白展堂:“是!”
白展堂:“兄台......
--------------------
李大嘴:“各回各家,各找各妈!”
白展堂:“回去!”
白展堂:“我知道了!
--------------------
李大嘴:“各回各家,各找各妈!”
白展堂:“我不是他们的人,我自己有本事做官,但谁也不能
--------------------


In [38]:
ret = sample(tokenizer, gpt, '李大嘴：“各回各家，各找各妈！” \n莫小贝：“', 3, 20, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

李大嘴:“各回各家,各找各妈!”
莫小贝:“我哪敢跟你说话,那是我爸妈,我......”
--------------------
李大嘴:“各回各家,各找各妈!”
莫小贝:“我也不知道我这个人,都是你说了算。”
--------------------
李大嘴:“各回各家,各找各妈!”
莫小贝:“大姐,你怎么就这么讨厌,这事儿我倒没想。”
--------------------


In [39]:
ret = sample(tokenizer, gpt, '李大嘴：“各回各家，各找各妈！” \n李白：“', 3, 20, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

李大嘴:“各回各家,各找各妈!”
李白:“咱们去看看,让那个⁇子手知道不。”
--------------------
李大嘴:“各回各家,各找各妈!”
李白:“行去!”
吕光:“去!”
--------------------
李大嘴:“各回各家,各找各妈!”
李白:“老李,你听我的!”
李白:“谁
--------------------


# 问答的例子

In [40]:
ret = sample(tokenizer, gpt, '中国的首都是北京\n日本的首都是东京\n美国的首都是', 3, 3, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

中国的首都是北京
日本的首都是东京
美国的首都是纽约
英国
--------------------
中国的首都是北京
日本的首都是东京
美国的首都是纽约
新加坡
--------------------
中国的首都是北京
日本的首都是东京
美国的首都是纽约
香港
--------------------


In [41]:
ret = sample(tokenizer, gpt, '李白所在朝代是唐\n李清照所在朝代是宋\n唐伯虎所在朝代是', 3, 1, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

李白所在朝代是唐
李清照所在朝代是宋
唐伯虎所在朝代是宋
--------------------
李白所在朝代是唐
李清照所在朝代是宋
唐伯虎所在朝代是宋
--------------------
李白所在朝代是唐
李清照所在朝代是宋
唐伯虎所在朝代是北宋
--------------------


In [42]:
ret = sample(tokenizer, gpt, '陈奕迅是歌星\n郭德纲是', 3, 1, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

陈奕迅是歌星
郭德纲是艺人
--------------------
陈奕迅是歌星
郭德纲是相声演员
--------------------
陈奕迅是歌星
郭德纲是相声演员
--------------------


In [43]:
ret = sample(tokenizer, gpt, '珠穆朗玛峰的高度（米）是', 3, 10, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

珠穆朗玛峰的高度(米)是世界最低的,海拔高度为778米
--------------------
珠穆朗玛峰的高度(米)是1260米,最高峰是2220米
--------------------
珠穆朗玛峰的高度(米)是5500米,高度(米)是
--------------------


In [44]:
ret = sample(tokenizer, gpt, '姚明的身高（米）是', 3, 10, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

姚明的身高(米)是多少?-体育
--------------------
姚明的身高(米)是有什么用?本题已加入知乎
--------------------
姚明的身高(米)是球员能够保持的最大高度。
--------------------


# 算数例子

In [45]:
ret = sample(tokenizer, gpt, '1+1=2\n2+2=4\n3+3=6\n4+4=', 3, 1, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

1+1=2
2+2=4
3+3=6
4+4=7
--------------------
1+1=2
2+2=4
3+3=6
4+4=7
--------------------
1+1=2
2+2=4
3+3=6
4+4=7
--------------------


In [46]:
ret = sample(tokenizer, gpt, '1+1=2\n1+2=3\n1+3=4\n1+4=', 3, 1, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

1+1=2
1+2=3
1+3=4
1+4=5
--------------------
1+1=2
1+2=3
1+3=4
1+4=5
--------------------
1+1=2
1+2=3
1+3=4
1+4=5
--------------------


# ???的例子

In [47]:
ret = sample(tokenizer, gpt, '''惊雷这通天修为
天塌地陷紫金锤
紫电这玄真火焰
''', 3, 30, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

惊雷这通天修为
天塌地陷紫金锤
紫电这玄真火焰
七种神通
一锤爆开
天崩地陷
天翻地陷
天崩地陷
天崩
--------------------
惊雷这通天修为
天塌地陷紫金锤
紫电这玄真火焰
天地灵兽
兵戈剑戟
战歌如歌战歌
大帝神剑
雷锋宝剑

--------------------
惊雷这通天修为
天塌地陷紫金锤
紫电这玄真火焰
化作了“九阳”
“九阳”
轰隆隆
“九阳”
--------------------


# 写作文例子

In [48]:
ret = sample(tokenizer, gpt, '''爱情是''', 3, 50, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

爱情是多么美好,对每个人都是多么的可贵。<eod>如何评价魅族官方声称会「上榜」?@魅族产品设计总监兼首席设计师@
--------------------
爱情是什么?我的第一个女朋友,比我大三岁,也比我小。她有个超级可爱的男朋友,而我只有初恋。她
--------------------
爱情是可以去学习的,但是你只有在学习了才有机会去恋爱,但是你对自己很负责,你有能力去选择如何去爱自己,而不是因为别人说什么就去改变
--------------------


In [49]:
ret = sample(
    tokenizer, gpt,
    '''一时黛玉进了荣府，下了车。众嬷嬷引着，便往东转弯，穿过一个东西的穿堂，向南大厅之后，仪门内大院落，上面五间大正房，两边厢房鹿顶耳房钻山，四通八达，轩昂壮丽，比贾母处不同。黛玉便知这方是正经正内室，一条大甬路，直接出大门的。''',
    3, 200, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)


一时黛玉进了荣府,下了车。众嬷嬷引着,便往东转弯,穿过一个东西的穿堂,向南大厅之后,仪门内大院落,上面五间大正房,两边厢房鹿顶耳房钻山,四通八达,轩昂壮丽,比贾母处不同。黛玉便知这方是正经正内室,一条大甬路,直接出大门的。黛玉便知,宝姐姐们已经住下了。这方正房内,放着凤姐、凤姐姐的黛玉梳妆台、扇子、茶杯、床。走到第二间大院落,门口有一牌楼,牌楼上的牌写的是:“黛玉睡在槛里”。这里,也是凤姐、凤姐姐的黛玉梳妆台、扇子、茶杯、床。进去,黛玉看见那扇子,便上前,说道:“小的们叫道:‘小的坐这里,你我这里坐!’”说着,便从怀里取出一张纸来,向宝玉的怀里一揣
--------------------
一时黛玉进了荣府,下了车。众嬷嬷引着,便往东转弯,穿过一个东西的穿堂,向南大厅之后,仪门内大院落,上面五间大正房,两边厢房鹿顶耳房钻山,四通八达,轩昂壮丽,比贾母处不同。黛玉便知这方是正经正内室,一条大甬路,直接出大门的。众人还不明白这是何事,先有人说:“这里角缝小些,待会儿有火盆子,就可以烧了。”黛玉道:“烧上,我就不进去了。”只见曹雪芹的正房三间,每间都有火盆子,黛玉便有了准备。王夫人道:“这里是我家主客厅,你先别进去,你有了铺盖,就把灯打开,说一声,道一声,‘好’。”宝玉听了,便在前头,道一声“好”。众人听了,皆侧目,道:“这才是好
--------------------
一时黛玉进了荣府,下了车。众嬷嬷引着,便往东转弯,穿过一个东西的穿堂,向南大厅之后,仪门内大院落,上面五间大正房,两边厢房鹿顶耳房钻山,四通八达,轩昂壮丽,比贾母处不同。黛玉便知这方是正经正内室,一条大甬路,直接出大门的。黛玉进来,进内室,见门是小门,里面有门,门里开着的是书案,旁边是书柜,里面是书架,架子上是几册《红楼梦》,几册《金瓶梅》,还有几册《脂砚斋重评红楼》。黛玉道:“看这书,也是小道儿写的。这书可得瞧瞧。”那里厢房里说道:“瞧瞧去。”黛玉便瞧那书架,说道:“有几卷,我却不知这书影。”便叫道:“你就在那里,快看。”那书影也跟着瞧了,不知怎么
--------------------


# 对话例子

In [50]:
ret = sample(
    tokenizer, gpt,
    '''A：“今天我想吃火锅”
B：“''',
    3, 50, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

A:“今天我想吃火锅”
B:“什么火锅”
C:“......”
D:“我想吃辣”
E:“我很喜欢辣”
F:“我喜
--------------------
A:“今天我想吃火锅”
B:“没事,就是我自己想吃火锅”
C:“我知道我一个人吃火锅的时候,会吃到很多不一样的味道,但是我没有这么说过”

--------------------
A:“今天我想吃火锅”
B:“不行,去吃火锅吧”
C:“去吃火锅吧”
D:“行,那我去吃麻辣烫吧”
E:“来
--------------------


In [51]:
ret = sample(
    tokenizer, gpt, '''A：“跟我一起去看电影吧”
B：“''',
    3, 50, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

A:“跟我一起去看电影吧”
B:“真的,我知道,我会带你去看电影的”
A:“好,那你怎么办?”
B:“我也不知道,我觉得
--------------------
A:“跟我一起去看电影吧”
B:“好的”
A:“......”
B:“你跟我走吧”
A:“......”
B:“......”
--------------------
A:“跟我一起去看电影吧”
B:“好”
A:“去了?”
B:“啊,真的”
A:“你什么意思”
B:“我想不出
--------------------


# 对联例子

In [52]:
duilian = '''
上联天，下联地
上联雨，下联风
上联大陆，下联长空
上联雷隐隐，下联雾蒙蒙
上联开心大吉，下联万事亨通
'''

In [53]:
ret = sample(
    tokenizer, gpt,
    duilian + '上联李白作诗，下联',
    5, 2, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)


上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联李白作诗,下联宋诗
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联李白作诗,下联安邦治
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联李白作诗,下联李白作诗
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联李白作诗,下联多情
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联李白作诗,下联笑颜
--------------------


In [54]:
ret = sample(
    tokenizer, gpt,
    duilian + '上联追求真爱，下联',
    5, 4, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)


上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联追求真爱,下联只爱我一人
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联追求真爱,下联万事兴
上
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联追求真爱,下联义无返顾
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联追求真爱,下联待你来
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联追求真爱,下联人人有福

--------------------


In [55]:
ret = sample(
    tokenizer, gpt,
    duilian + '上联天天向上，下联',
    5, 4, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)


上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联天天向上,下联颂一声

--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联天天向上,下联天天平安
上
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联天天向上,下联人人幸福
上
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联天天向上,下联兴旺发达

--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联天天向上,下联小赌怡情
--------------------


In [56]:
ret = sample(
    tokenizer, gpt,
    duilian + '上联天地在我心，下联',
    5, 4, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)


上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联天地在我心,下联欢天喜地
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联天地在我心,下联法力腾
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联天地在我心,下联万事如意

--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联天地在我心,下联鹏程万里
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联天地在我心,下联我自在逍遥
--------------------


# 名人名言

In [57]:
ret = sample(
    tokenizer, gpt, '''鲁迅曾经说过：“''',
    3, 50, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

鲁迅曾经说过:“‘不讲这一套,我们就没有事情做。’我的意思是,要想到自己的使命,不讲一套,我们就什么都没有做。”
--------------------
鲁迅曾经说过:“不信上帝的人,死也不能复活。”可见耶稣和耶稣教堂、耶稣会堂、天主教堂、犹太教堂等基督教堂的地位实在是远远超越了犹太
--------------------
鲁迅曾经说过:“‘观者莫若观,观者莫若观’”。在《新唐书·玄宗纪》中,李辅国就说:“昔
--------------------


In [58]:
ret = sample(
    tokenizer, gpt, '''爱因斯坦曾经说过：“''',
    3, 50, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

爱因斯坦曾经说过:“科学家的使命在于发现真理。”而科学家的使命,恰恰是探索真相。如果不了解真实的世界,科学家就无法探索出真实的宇宙。
--------------------
爱因斯坦曾经说过:“我对过去的事情并不了解,我不知道以后会发生什么。”如果你问:“你有可能在任何时候去自杀,这是真的吗?”
--------------------
爱因斯坦曾经说过:“你在艺术家面前是个骗子。”正如他在一次著名的演讲中所说:“你很少在艺术家面前提起艺术家。”为什么说
--------------------


In [59]:
ret = sample(
    tokenizer, gpt, '''牛顿曾经说过：“''',
    3, 50, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

牛顿曾经说过:“我们在这个世界上,‘生活在’在这个世界上。”但事实是,这并不是我们的错。在这个世界上,我们不应该
--------------------
牛顿曾经说过:“我们的祖先就知道了动物,因此我们只能说那些动物不知道的事。”但是,我们的祖先也不知道。达尔文的进化论(达尔文进化论)
--------------------
牛顿曾经说过:“人类社会是一种大灾难,它是地球对太阳的‘保护’。地球上的‘大灾难’是一个‘大
--------------------


In [60]:
ret = sample(
    tokenizer, gpt, '''佛祖曾经说过：“''',
    3, 50, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

佛祖曾经说过:“不信,那种书,可以随便看看。”我就找了一本,看了十几页,还不认识,只有一个字:“完!”我
--------------------
佛祖曾经说过:“佛祖的话,虽然好像有些微言大义,其实也就那样。”可见,佛祖教我们做人,既要信则有信,不信则无,
--------------------
佛祖曾经说过:“有了德性,没有了智性,无论多么贫瘠的土地,都有了温暖的、令人满意的天空。”说这句话时,也不忘了提醒
--------------------


In [61]:
ret = sample(
    tokenizer, gpt, '''乔布斯曾经说过：“''',
    3, 50, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

乔布斯曾经说过:“美国人是不懂得他们的文化的。”[21]在此情况下,威廉·阿伦特在解释自己的观点时说:“美国人比我们
--------------------
乔布斯曾经说过:“那些话不是人说的,是说你的。”我也是从这里学会了一些该怎么做。接下来要做的就是如何去面对恐惧。
--------------------
乔布斯曾经说过:“我们可以找到一种方法,就是用自己的语言将自己的思想通过对方的语言表达出来。”这个方法就是语言学习的关键。当你的词汇量达到一定程度
--------------------
